In [ ]:
from flask import Flask, request, render_template
import joblib
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')

app = Flask(__name__)

# Load models and vectorizers
LR = joblib.load("logreg_model.pkl")
lr = joblib.load("linreg_model.pkl")
vectorization = joblib.load("logreg_vectorizer.pkl")
tfidf = joblib.load("linreg_vectorizer.pkl")

stemmer = SnowballStemmer("english")
stopword = set(stopwords.words('english'))

def text_cleaning(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = " ".join([stemmer.stem(word) for word in text.split() if word not in stopword])
    return text

def output_label(n):
    return "Not A Fake Review" if n == 1 else "Fake Review"

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        review = request.form["review"]
        cleaned_review = text_cleaning(review)

        log_feat = vectorization.transform([cleaned_review])
        lin_feat = tfidf.transform([cleaned_review])

        log_pred = LR.predict(log_feat)
        log_bin = 0 if log_pred[0] == 'CG' else 1

        lin_val = lr.predict(lin_feat)[0]
        lin_bin = 1 if lin_val >= 0.5 else 0

        return render_template("index.html",
                               review=review,
                               logistic_result=output_label(log_bin),
                               linear_result=output_label(lin_bin))
    return render_template("index.html")

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


[nltk_data] Downloading package stopwords to C:\Users\KRISHNA
[nltk_data]     CHAITHANYA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [18/May/2025 12:22:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2025 12:22:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/May/2025 12:23:30] "POST / HTTP/1.1" 200 -
